In [3]:
import requests
from bs4 import BeautifulSoup as BS, element as BS_element
import lxml
import pandas as pd

In [4]:
rutor_url = 'http://5.45.84.133/new/'
host = '/'.join(rutor_url.split('/', maxsplit=3)[:3])
response = requests.get(rutor_url)
response.status_code

200

In [5]:
soup = BS(response.content, 'lxml')
main_table = soup.find('div', id='index')
dependent_tables: tuple[BS_element.Tag] = tuple(main_table.find('h2').find_next_siblings())

In [6]:
def _get_from_table(table: BS_element.Tag, group_name: str) -> pd.DataFrame:
    elements: tuple[BS_element.Tag] = tuple(table.find_all('td'))
    BS_data, BS_name, BS_size, BS_pirs = elements[0],elements[1],elements[-2],elements[-1]
    
    data = BS_data.get_text(strip=True)
    
    BS_name: BS_element.Tag = tuple(BS_name.find_all('a'))[-1]
    name = BS_name.get_text(strip=True)
    href = host + BS_name.get('href')

    size = BS_size.get_text(strip=True)
    
    up, down = tuple(map(int, BS_pirs.stripped_strings))
    
    return pd.DataFrame({
        'group': group_name,
        'data': data,
        'name': name,
        'size': size,
        'up': up,
        'down': down,
        'href': href
    }, index=[0])
    
    

def _get_table(elements: tuple[BS_element.Tag, BS_element.Tag]) -> list[pd.DataFrame]:
    bs_name, bs_table = elements
    group_name = bs_name.find('a').get_text(strip=True)
    group_tables: tuple[BS_element.Tag] = tuple(bs_table.find('tr', class_='backgr').find_next_siblings('tr'))
    
    ap_dataframe = []
    for item in group_tables:
        ap_dataframe.append(_get_from_table(item, group_name))
    
    return ap_dataframe
    
    
rearranged_tables: tuple[tuple[BS_element.Tag, BS_element.Tag]] = tuple(zip(dependent_tables[0::2], dependent_tables[1::2]))
ap_dataframe = []
for item in rearranged_tables:
    ap_dataframe.append(_get_table(item))

ap_dataframe = sum(ap_dataframe, [])
df = pd.concat(ap_dataframe, ignore_index=True, sort= False)

In [7]:
test = df['group'].unique()
print(test)
interesting = test[[0, 3, 6, 7, 10]]

['Зарубежные фильмы' 'Наши фильмы' 'Научно-популярные фильмы'
 'Зарубежные сериалы' 'Наши сериалы' 'Телевизор' 'Мультипликация' 'Аниме'
 'Иностранные релизы' 'Музыка' 'Игры' 'Софт' 'Спорт и Здоровье' 'Юмор'
 'Хозяйство и Быт' 'Книги' 'Другое']


In [10]:
my_df = df[df['group'].isin(interesting)]

cut = 30
my_df = my_df[my_df['up'] >= cut]
pd.set_option('display.max_colwidth', 1000)
display(my_df[my_df.columns[:-1]])

,group,data,name,size,up,down
0,Зарубежные фильмы,09 Май 23,Восстание зловещих мертвецов / Evil Dead Rise (2023) WEB-DLRip от New-Team | A | Есарев,1.46 GB,118,42
5,Зарубежные фильмы,09 Май 23,Переводчик / Guy Ritchie's the Covenant / The Covenant (2023) WEB-DLRip-AVC от DoMiNo | Sub,1.97 GB,40,42
8,Зарубежные фильмы,09 Май 23,Восстание зловещих мертвецов / Evil Dead Rise (2023) WEB-DLRip 720p от DoMiNo | Sub,1.78 GB,86,54
10,Зарубежные фильмы,09 Май 23,Рулетка смерти / Uno Para Morir / Deaths Roulette (2023) WEB-DLRip от toxics | L,1.46 GB,98,64
11,Зарубежные фильмы,09 Май 23,Переводчик / Guy Ritchie's the Covenant / The Covenant (2023) WEB-DL 1080p | Sub,6.22 GB,462,487
14,Зарубежные фильмы,09 Май 23,Восстание зловещих мертвецов / Evil Dead Rise (2023) WEB-DL 1080p | Sub,5.86 GB,457,199
18,Зарубежные фильмы,09 Май 23,Человек-муравей и Оса: Квантомания / Ant-Man and the Wasp: Quantumania (2023) HDRip-AVC от DoMiNo & селезень | D | MovieDalen,745.68 MB,39,19
21,Зарубежные фильмы,09 Май 23,Человек-муравей и Оса: Квантомания / Ant-Man and the Wasp: Quantumania (2023) HDRip-AVC от DoMiNo & селезень | D,2.24 GB,127,51
27,Зарубежные фильмы,09 Май 23,Человек-муравей и Оса: Квантомания / Ant-Man and the Wasp: Quantumania (2023) HDRip от MegaPeer | D | MovieDalen,1.46 GB,93,36
28,Зарубежные фильмы,09 Май 23,"Мой ужасный сосед / A Man Called Otto (2022) BDRip 1080p от ExKinoRay | D, P",13.67 GB,37,45


In [11]:
id = [11]
my_df.loc[id, 'href']

11    http://5.45.84.133/torrent/923796/perevodchik_guy-ritchies-the-covenant_the-covenant-2023-web-dl-1080p-sub
Name: href, dtype: object